# This notebook will populate the .ini files with the injection parameters

In [2]:
import bilby as bb
import numpy as np
import gwcosmo as gwc
import pandas as pd
import GCR
import GCRCatalogs as GCRCat
import matplotlib.pyplot as plt
import os
from astropy.time import Time

In [6]:
nsbhIniTemplatePath = "/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data/slurmJobs/prod/template_acceptance-walk_prod_NSBH_beforeModifications.ini"
bbhIniTemplatePath = "/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data/slurmJobs/prod/template_acceptance-walk_prod_BBH_beforeModifications.ini"

## The following should be appended to the existing template .ini files:
- `label`: best to use the galaxyID_iterator based on some galaxies having multiple CBC's
- `outdir`: `os.path.join(basePath,label)`
- `injection_dict`: pulled directly from the entry in the CBC catalog. Note that the keys needed are 'mass_ratio', 'chirp_mass', 'luminosity_distance', 'dec','ra', 'theta_jn', 'psi', 'phase', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl', and 'geocent_time'. Note that geocent time should be set to zero

In [51]:
basePathBBH = "/pscratch/sd/s/seanmacb/BBH_GalaxyIDs"
basePathNSBH = "/pscratch/sd/s/seanmacb/NSBH_GalaxyIDs"

- Set `outdir` to basePath[NSBH.BBH]/batch_[0..11]/{galaxyID_iterator}
- Set `label` to `galaxyID_iterator`

In [88]:
def append_to_template(template_path, new_lines, output_path):
    """
    Read a template file, append lines, and write to a new file.

    Parameters:
    - template_path (str): Path to the original template file.
    - new_lines (list of str): Lines to append to the file.
    - output_path (str): Path to save the new file.
    """
    # Read the original template
    with open(template_path, 'r') as file:
        content = file.read()

    # Add the new lines
    appended_content = content + '\n' + '\n'.join(new_lines) + '\n'

    # Write the new content to the output file
    with open(output_path, 'w') as file:
        file.write(appended_content)
    return output_path

def formatLines(galaxyRow,basePath,df):
    """
    Read the galaxyRow, basePath, the input dataframe, and returns a list of formatted lines that match the .ini syntax

    Parameters:
    - galaxyRow (Pandas Series): The row entry of the CBC in question, taken directly from the mock CBC catalog
    - basePath (str): The base path to the directory where the ini file will be saved
    - df (Pandas dataframe): The dataframe of the mock CBC catalog
    """
    keys = ['mass_ratio', 'chirp_mass', 'luminosity_distance', 'dec','ra', 'theta_jn', 'psi', 'phase', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl', 'geocent_time'] # Injection keys
    dfKeys = ['mass_ratio', 'chirp_mass', 'luminosity_distance', 'mdec','mra', 'theta_jn', 'psi', 'phase', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl', 'geocent_time'] # Dataframe keys

    # Format the first line, related to label
    galaxyID = int(galaxyRow["galaxyID"])

    a = np.sort(df[df["galaxyID"]==galaxyRow["galaxyID"]].index)
    iterator = np.argwhere(a==galaxyRow.name)[0][0]
    
    line1="label={}_{}".format(galaxyID,iterator)

    # Format the second line, related to the outdir
    line2="outdir={}/{}_{}".format(basePath,galaxyID,iterator)

    # Format the third line, related to the injection dict
    # indexedRow = galaxyRow[dfKeys]

    injection_dict = {}

    for k1,k2 in zip(keys,dfKeys):
        try:
            injection_dict[k1]=testRow[k2]
        except:
            injection_dict[k1]=0

    line3 = "injection_dict={}".format(injection_dict)

    return line1,line2,line3